In [ ]:
import random
import copy
import time
import pandas as pd
import numpy as np
import gc
import re
import torch
import string
from keras.preprocessing.text import Tokenizer
#import spacy
from tqdm import tqdm_notebook, tnrange
from tqdm.auto import tqdm

tqdm.pandas(desc='Progress')
from collections import Counter

from nltk import word_tokenize

import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
from torch.utils.data import Dataset, DataLoader
from torch.nn.utils.rnn import pack_padded_sequence, pad_packed_sequence
from torch.autograd import Variable
from sklearn.metrics import f1_score
import os
from keras.utils import pad_sequences

# cross validation and metrics
from sklearn.model_selection import StratifiedKFold
from sklearn.metrics import f1_score
from torch.optim.optimizer import Optimizer

from sklearn.preprocessing import StandardScaler
from multiprocessing import  Pool
from functools import partial
import numpy as np
from sklearn.decomposition import PCA
import torch as t
import torch.nn as nn
import torch.nn.functional as F

import matplotlib.pyplot as plt
df=pd.read_csv("/content/train.csv")
# Remove column name 'text_id'
df.drop(df.columns[[0]], axis=1, inplace=True)
df.head()
df.info()
def process_data(df , column):
    # Remove punctuation
    punct = set(string.punctuation)
    # tokens = [token for token in tokens if token not in punct]
    df[column]=df[column].apply(lambda x: ''.join(ch for ch in x if ch not in punct))

    #Remove special characters
    df[column] = df[column].str.replace('[^\w\s]',' ')

    # Lowercase
    # tokens = [token.lower() for token in tokens]
    df[column] = df[column].apply(lambda x: x.lower())

    # Remove \n
    # tokens = [token.replace('\n',' ') for token in tokens]
    df[column] = df[column].apply(lambda x: x.replace('\n',''))

    # Remove ordinal number patterns
    df[column] = df[column].str.replace(r'\d+(?:st|nd|rd|th)', '')

    # Remove all numbers from text
    # Clean numbers from text column
    df[column] = df[column].str.replace('\d+', '')

    #Remove numbers
    from string import digits
    remove_digits = str.maketrans('','',digits)
    df[column] = df[column].apply(lambda x: x.translate(remove_digits))

    #Remove extra space
    import re
    df[column] = df[column].apply(lambda x: x.strip())
    df[column] = df[column].apply(lambda x: re.sub(" +", " ", x))

    return(df)
df = process_data(df, 'sentence')
df.head()
import nltk
nltk.download('wordnet')
from nltk.stem import WordNetLemmatizer
lemmatizer = WordNetLemmatizer()
def lemmatize_words(text):
    words = text.split()
    words = [lemmatizer.lemmatize(word,pos='v') for word in words]
    return ' '.join(words)
df['sentence'] = df['sentence'].apply(lemmatize_words)
df.head()
from sklearn.model_selection import train_test_split
import numpy as np
train_df, val_df = train_test_split(df, test_size=0.20)
print("Train shape : ",train_df.shape)
print("Test shape : ",val_df.shape)
train_X=train_df['sentence']
max_features = 50000
tokenizer = Tokenizer(num_words=max_features)
tokenizer.fit_on_texts(list(train_X))
train_X = tokenizer.texts_to_sequences(train_X)
val_X = tokenizer.texts_to_sequences(val_df['sentence'])
maxlen = 180
train_X = pad_sequences(train_X, maxlen=maxlen)
val_X = pad_sequences(val_X, maxlen=maxlen)
train_df['neutral']=train_df['gold_label']==0
train_df['negative']=train_df['gold_label']==-1
train_df['positive']=train_df['gold_label']==1
train_df['neutral']=train_df['neutral'].astype(float)
train_df['negative']=train_df['negative'].astype(float)
train_df['positive']=train_df['positive'].astype(float)
val_df['neutral']=val_df['gold_label']==0
val_df['negative']=val_df['gold_label']==-1
val_df['positive']=val_df['gold_label']==1
val_df['neutral']=val_df['neutral'].astype(float)
val_df['negative']=val_df['negative'].astype(float)
val_df['positive']=val_df['positive'].astype(float)
train_df.head()
train_y = (train_df[['negative','neutral','positive']].values)
val_y = (val_df[['negative','neutral','positive']].values)
embed_size = 128 # how big is each word vector
batch_size = 128 # how many samples to process at once
n_splits = 4 # Number of K-fold Splits
SEED = 123
debug = 0
class BiLSTMModel(nn.Module):

    def __init__(self, hidden_size, num_layers, n_classes, dropout=0.1, pretrained_embeddings=None):
        super(BiLSTMModel, self).__init__()
        self.embedding = nn.Embedding(max_features, embed_size)
        # self.bilstm = nn.LSTM(input_size=embed_size, hidden_size=hidden_size, num_layers=num_layers, batch_first=True, bidirectional=True)
        self.bilstm1 = nn.LSTM(input_size=embed_size, hidden_size=hidden_size, num_layers=num_layers, batch_first=True, bidirectional=True)
        self.bilstm2 = nn.LSTM(input_size=hidden_size*2, hidden_size=hidden_size, num_layers=num_layers, batch_first=True, bidirectional=True)
        self.dropout = nn.Dropout(dropout)
        self.fc1 = nn.Linear(hidden_size*2, hidden_size)
        self.fc2 = nn.Linear(hidden_size, n_classes)

    def forward(self, x):
        x = self.embedding(x)
        output, _ = self.bilstm1(x)
        output, _ = self.bilstm2(output)
        output = self.dropout(output)
        output = torch.mean(output, dim=1)
        output = self.dropout(output)
        output = self.fc1(output)
        output = self.dropout(output)
        output = self.fc2(output)
        return output
# class LSTMModel(nn.Module):

#     def __init__(self, hidden_size, num_layers, n_classes, dropout=0.1):
#         super(LSTMModel, self).__init__()
#         self.embedding = nn.Embedding(max_features, embed_size)
#         self.rnn = nn.LSTM(input_size=embed_size, hidden_size=hidden_size, num_layers=num_layers, batch_first=True, bidirectional=False)
#         self.dropout = nn.Dropout(dropout)
#         self.fc1 = nn.Linear(hidden_size, n_classes)

#     def forward(self, x):
#         x = self.embedding(x)
#         output, _ = self.rnn(x)
#         output = self.dropout(output)
#         output = torch.mean(output, dim=1)
#         output = self.dropout(output)
#         output = self.fc1(output)
#         return output

# class ImprovedRNN(nn.Module):

#     def __init__(self, hidden_size, num_layers, n_classes, dropout=0.1):
#         super(ImprovedRNN, self).__init__()
#         self.embedding = nn.Embedding(max_features, embed_size)
#         self.rnn = nn.LSTM(input_size=embed_size, hidden_size=hidden_size, num_layers=num_layers, batch_first=True, bidirectional=True)
#         self.attention = nn.MultiheadAttention(embed_dim=hidden_size*2, num_heads=8)
#         self.dropout = nn.Dropout(dropout)
#         self.fc1 = nn.Linear(hidden_size*2, n_classes)

#     def forward(self, x):
#         x = self.embedding(x)
#         output, _ = self.rnn(x)
#         output = self.dropout(output)
#         output, _ = self.attention(output.transpose(0, 1), output.transpose(0, 1), output.transpose(0, 1))
#         output = output.transpose(0, 1)
#         output = torch.mean(output, dim=1)
#         output = self.dropout(output)
#         output = self.fc1(output)
#         return output

# class RNN_Text(nn.Module):

#     def __init__(self):
#         super(RNN_Text, self).__init__()
#         hidden_size = 64
#         num_layers = 4
#         n_classes = 3
#         self.embedding = nn.Embedding(max_features, embed_size)
#         self.rnn = nn.LSTM(input_size=embed_size, hidden_size=hidden_size, num_layers=num_layers, batch_first=True)
#         self.fc1 = nn.Linear(hidden_size, n_classes)

#     def forward(self, x):
#         x = self.embedding(x)
#         output, hidden = self.rnn(x)
#         x = output[:, -1, :]  # take the final hidden state
#         x = self.fc1(x)
#         return x

# class CNN_Text(nn.Module):

#     def __init__(self):
#         super(CNN_Text, self).__init__()
#         filter_sizes = [1,2,3,5]
#         num_filters = 40
#         n_classes = 3
#         self.embedding = nn.Embedding(max_features, embed_size)
#         self.convs1 = nn.ModuleList([nn.Conv2d(1, num_filters, (K, embed_size)) for K in filter_sizes])
#         self.dropout = nn.Dropout(0.1)
#         self.fc1 = nn.Linear(len(filter_sizes)*num_filters, n_classes)


#     def forward(self, x):
#         x = self.embedding(x)
#         x = x.unsqueeze(1)
#         x = [F.relu(conv(x)).squeeze(3) for conv in self.convs1]
#         x = [F.max_pool1d(i, i.size(2)).squeeze(2) for i in x]
#         x = torch.cat(x, 1)
#         x = self.dropout(x)
#         logit = self.fc1(x)
#         return logit
n_epochs = 5
model = BiLSTMModel(hidden_size=128, num_layers=4, n_classes=3)
loss_fn = nn.CrossEntropyLoss(reduction='sum')
optimizer = torch.optim.Adam(filter(lambda p: p.requires_grad, model.parameters()), lr=0.001)


# Load train and test in Memory
x_train = torch.tensor(train_X, dtype=torch.long)
y_train = torch.tensor(train_y, dtype=torch.long)
x_cv = torch.tensor(val_X, dtype=torch.long)
y_cv = torch.tensor(val_y, dtype=torch.long)

# Create Torch datasets
train = torch.utils.data.TensorDataset(x_train, y_train)
valid = torch.utils.data.TensorDataset(x_cv, y_cv)

# Create Data Loaders
train_loader = torch.utils.data.DataLoader(train, batch_size=batch_size, shuffle=True)
valid_loader = torch.utils.data.DataLoader(valid, batch_size=batch_size, shuffle=False)

train_loss = []
valid_loss = []

for epoch in range(n_epochs):
    start_time = time.time()
    # Set model to train configuration
    model.train()
    avg_loss = 0.
    for i, (x_batch, y_batch) in enumerate(train_loader):
        # Predict/Forward Pass
        y_pred = model(x_batch)
        # Compute loss
        loss = loss_fn(y_pred, y_batch.float())
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        avg_loss += loss.item() / len(train_loader)

    # Set model to validation configuration -Doesn't get trained here
    model.eval()
    avg_val_loss = 0.
    val_preds = np.zeros((len(x_cv),3))

    for i, (x_batch, y_batch) in enumerate(valid_loader):
        y_pred = model(x_batch).detach()
        avg_val_loss += loss_fn(y_pred, y_batch.float()).item() / len(valid_loader)
        # keep/store predictions
        val_preds[i * batch_size:(i+1) * batch_size] =F.softmax(y_pred).cpu().numpy()

    # Check Accuracy
    val_accuracy = sum(val_preds.argmax(axis=1)==val_y.argmax(axis=1))/len(val_y)
    train_loss.append(avg_loss)
    valid_loss.append(avg_val_loss)
    elapsed_time = time.time() - start_time
    print('Epoch {}/{} \t loss={:.4f} \t val_loss={:.4f}  \t val_acc={:.4f}  \t time={:.2f}s'.format(
                epoch + 1, n_epochs, avg_loss, avg_val_loss, val_accuracy, elapsed_time))
dev_df = pd.read_csv("/content/test.csv")
dev_df = process_data(dev_df, 'sentence')
test_X = tokenizer.texts_to_sequences(dev_df['sentence'].astype(str))
test_X = pad_sequences(test_X, maxlen=maxlen)
x_cv = torch.tensor(test_X, dtype=torch.long)
valid = torch.utils.data.TensorDataset(x_cv)
valid_loader = torch.utils.data.DataLoader(valid, batch_size=64, shuffle=False)
model.eval()
y_hat=[]
batch_size=64
val_preds = np.zeros((len(x_cv),3))
for i, (x_batch) in enumerate(valid_loader):
        #print(x_batch[0])
        y_pred = model(x_batch[0]).detach()
        val_preds[i * batch_size:(i+1) * batch_size] =F.softmax(y_pred).cpu().numpy()
y_hat=(val_preds.argmax(axis=1))
k=0
with open('/content/answer.txt', 'w') as f:
    for i in (y_hat-1):
        f.write(str(i))
        k+=1
        if k==4855:
          break
        f.write('\n')


In [ ]:
len(dev_df)==4855